 ##  Streamed vs. Radio

## Introduction

This project focuses on popularity trends within the music industry. Specifically, this study examines the variation in radio and streaming music preferences from 2013 to 2022. Using the Billboard APIs, we can observe correlation between streamed music and radio music. Further, using the Genius API, we will look at the number of unique words in radio music versus streamed music. I am interested in whether or not the vocabulary of radio music and streamed songs have progressed at a similar rate during the last decade. 

## Hypothesis

My hypothesis is that streaming preferences were once in alignment with the songs on the radio, but this has occurred less and less over time. Factors responsible for this change could include an increase in streaming availability, making all types of music more accessible to audiences.

## APIs

This project uses the Billboard and LyricGenius APIs. 

In [ ]:


# Import billoard and genius APIs
import billboard
import lyricsgenius

#Import Python tools for handling numbers and statistics
import numpy
import statistics

# Using my unique genius API key. 
genius = lyricsgenius.Genius("R68yfEAxTTnQh-X7dkseqDysn96StA92nZJQn6gqy5JrZhmQbm6kWZvT7WH22s1s")

print("Percent of top streaming songs matching top radio songs")

# start of a delimited output report, header
report = "year\tpercent radio matching streaming\tstreaming avg vocab\tradio avg vocab\tstreaming median vocab\tradio median vocab\n"

# this is the available range of data from billboard API, 2013-2022
for year in range(2013, 2022):
    # get the chart of top streaming songs
    chartStreaming = billboard.ChartData('streaming-songs', year=year)
    
    # get the chart of top radio songs
    chartRadio = billboard.ChartData('radio-songs', year=year)
    
    # keep track of unqiue words per song
    wordsStreaming = []
    wordsRadio = []

    songsStreaming = []
    print("processing streaming ", year)
    # get the lyrics for each of the songs returned above, retry if API fails
    for song in chartStreaming: # for each song in chartStreaming
        lyric = None 
        keepTryingGenius = True
        retryCount = 20
        while keepTryingGenius == True and retryCount > 0:
            try:
                lyric = genius.search_song(song.title, song.artist) # get the lyrics to the song
                keepTryingGenius = False # stop even if lyric is empty (not found)
            except:
                print("We need to retry ", song.title, " by ", song.artist)
                retryCount = retryCount - 1
        if lyric != None:
            count = len(set(lyric.lyrics.split())) # count the unique words in song lycrics
            wordsStreaming.append(count) # append the count to the list of streaming words
        songsStreaming.append(song.title) # in theory titles with different artists could match; in practice, not

    # same as above, but for radio
    # a generic function could be written to reduce duplication
    songsRadio = []
    print("processing radio ", year)
    for song in chartRadio:
        lyric = None 
        keepTryingGenius = True
        retryCount = 20
        if(song.title.startswith("2017")): # the 2017 grammys is not a single song. ignore it.
            keepTryingGenius = False # ignore a series of grouped songs, special case, not relevant
        while keepTryingGenius == True and retryCount > 0:
            try:
                lyric = genius.search_song(song.title, song.artist)
                keepTryingGenius = False # stop even if lyric is empty (not found)
            except:
                print("We need to retry ", song.title, " by ", song.artist)
                retryCount = retryCount - 1
        if lyric != None:
            count = len(set(lyric.lyrics.split()))
            wordsRadio.append(count)
        songsRadio.append(song.title)

    d = [value for value in songsStreaming if value in songsRadio] # d becomes count of matching songs between radio and streaming per year
    percentMatch = len(d) / len(songsRadio) # calculate percent match instead of absolute number
    # add to report
    report = report + str(year) + "\t" + str(len(d)) + "%\t" +  str(numpy.average(wordsStreaming)) + "\t" + str(numpy.average(wordsRadio)) + "\t" + str(statistics.median(wordsStreaming)) + "\t" + str(statistics.median(wordsRadio)) + "\n"

# when loop is complete processing all years, print the report
print(report)





ModuleNotFoundError: No module named 'billboard'

![Radio Streaming Percent Match](radio_streaming_match.jpg) ![Streaming_Vs_Radio_Vocab](streaming_vs_radio_vocab.jpg)


## Conclussion 

These results indicate that people once streamed similar music to what was played on the radio, but this pattern has been occurring less and less from 2013 to 2022. Further, the vocabulary of streamed music versus radio was once the same, but the number of unique words in streaming music has drastically increased, while the number of unique words in radio music has remained static. 

These results leave me with more significant questions, such as why the number of unique words in streaming music continues to grow. Perhaps radio is meant for a wide general audience, making it impossible for the preferences of all audience members to be exemplified. Has pandemic lockdown impacted the time that people spend listening to the radio, making listeners more likely to stream new songs? Streaming versus radio preferences began to take a big shift in 2014, indicating that this hypothesis could not be the only factor responsible for the relationship between radio and streaming.
